El código realiza una serie de operaciones de procesamiento y partición de datos utilizando las bibliotecas `pandas` y `numpy`. Aquí está el resumen de las acciones principales que realiza el código:

1. **Carga de Datos**: 
   - Se importan las bibliotecas `numpy` y `pandas`.
   - Se carga un archivo CSV (`datos_limpiados.csv`) en un DataFrame llamado `datos`.

2. **Preparación de Datos**:
   - Se establece la columna 'Time' como el índice del DataFrame y se convierte a un índice de fecha y hora.
   - Se eliminan las columnas no numéricas, en este caso, la columna 'Date'.
   - Los datos se resamplean a nivel horario tomando la media de cada hora.

3. **Limpieza de Datos**:
   - Se elimina la última fila del DataFrame.
   - Se verifica si hay valores vacíos en el DataFrame.
   - Se completan los valores vacíos utilizando la técnica de 'forward fill' y 'backward fill'.
   - Se verifica nuevamente si hay valores vacíos después de la imputación.

4. **Guardar Datos Preprocesados**:
   - Los datos readaptados se guardan en un nuevo archivo CSV (`datos_readaptados.csv`).

5. **Partición de Datos**:
   - Se seleccionan datos dentro de un rango de fechas específico (desde '2012-01-01 00:00:00' hasta '2014-12-30 23:00:00').
   - Se calcula el tamaño total de los datos y se divide en tres conjuntos: entrenamiento (70%), validación (15%) y prueba (15%).
   - Se determinan las fechas límite para cada conjunto ajustando las horas a las 23:00 horas para evitar divisiones a mitad del día.
   - Se crean particiones de datos para entrenamiento, validación y prueba.
   - Se eliminan las primeras entradas de los conjuntos de validación y prueba para evitar solapamiento.

6. **Conversión de Tipos de Datos**:
   - Se mapean valores específicos en la columna 'Holiday' a valores booleanos (0.0 a False y 1.0 a True).
   - Se asegura la tipificación booleana de la columna 'Holiday'.

7. **Verificación y Guardado de Particiones**:
   - Se muestran los tamaños de los conjuntos de entrenamiento, validación y prueba.
   - Se imprimen las fechas límite para verificación.
   - Se muestran las últimas filas del conjunto de entrenamiento y validación, y las primeras filas del conjunto de prueba.
   - Finalmente, cada conjunto se guarda en un archivo CSV separado (`train.csv`, `val.csv` y `test.csv`).

El código asegura que los datos están adecuadamente preparados y particionados para ser utilizados en un modelo de aprendizaje automático o análisis posterior.

In [37]:
import numpy as np
import pandas as pd
datos=pd.read_csv(r"C:\Users\arzua\OneDrive\Escritorio\preelec\src\data\datos_limpiados.csv")

#establecer time como indice

datos = datos.set_index('Time')
datos.index = pd.to_datetime(datos.index)
# Seleccionar solo las columnas numéricas
datos = datos.drop(columns='Date')

# Resamplear los datos a nivel horario sin introducir información futura
datos= datos.resample('H', closed='left', label='right').mean()

datos

#eliminacion del ultimo
datos=datos.iloc[:-1]
datos
# Comprobar si hay valores vacíos
print("¿Hay valores vacíos en el dataset?", datos.isnull().values.any())

# Completar valores vacíos con la media ponderada (forward fill y backward fill)
datos = datos.fillna(method='ffill').fillna(method='bfill')

# Verificar de nuevo
print("¿Hay valores vacíos en el dataset después de completar?", datos.isnull().values.any())
#guardar datos readaptados finales
datos.to_csv(r"C:\Users\arzua\OneDrive\Escritorio\preelec\src\data\datos_readaptados.csv")
import pandas as pd

# Suponiendo que 'datos' es un DataFrame con un índice de fecha y hora
datos = datos.loc['2012-01-01 00:00:00':'2014-12-30 23:00:00'].copy()

# Calcular el tamaño de cada conjunto
total_size = len(datos)
train_size = int(0.70 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

# Obtener las fechas límite basadas en el tamaño calculado, ajustando a las 23:00 horas
fin_train = datos.index[train_size - 1].replace(hour=23)
fin_val = datos.index[train_size + val_size - 1].replace(hour=23)

# Crear las particiones de datos
datos_train = datos.loc[:fin_train].copy()
datos_val = datos.loc[fin_train:fin_val].copy()
datos_test = datos.loc[fin_val:].copy()

# Eliminar la primera entrada de los conjuntos de validación y prueba
datos_val = datos_val.iloc[1:].copy()
datos_test = datos_test.iloc[1:].copy()

# Mostrar tamaños de los conjuntos
print("Tamaño del conjunto de entrenamiento:", len(datos_train))
print("Tamaño del conjunto de validación:", len(datos_val))
print("Tamaño del conjunto de prueba:", len(datos_test))

# Mostrar las fechas límite para verificación
print("Fecha fin entrenamiento:", fin_train)
print("Fecha fin validación:", fin_val)

# Mapear valores específicos a booleanos (suponiendo 0.0 y 1.0 como False y True respectivamente)
datos_train['Holiday'] = datos_train['Holiday'].map({0.0: False, 1.0: True})
datos_val['Holiday'] = datos_val['Holiday'].map({0.0: False, 1.0: True})
datos_test['Holiday'] = datos_test['Holiday'].map({0.0: False, 1.0: True})

# Convertir a booleano (esto es redundante ya que el mapeo ya lo convierte a bool, pero se deja para asegurar la tipificación)
datos_train['Holiday'] = datos_train['Holiday'].astype(bool)
datos_val['Holiday'] = datos_val['Holiday'].astype(bool)
datos_test['Holiday'] = datos_test['Holiday'].astype(bool)

# Mostrar los primeros registros para verificar
print(datos_train.tail())
print(datos_val.tail())
print(datos_test.head())

# Guardar cada conjunto en un archivo CSV separado
datos_train.to_csv(r"C:\Users\arzua\OneDrive\Escritorio\preelec\src\data\train.csv")
datos_val.to_csv(r"C:\Users\arzua\OneDrive\Escritorio\preelec\src\data\val.csv")
datos_test.to_csv(r"C:\Users\arzua\OneDrive\Escritorio\preelec\src\data\test.csv")

C:\Users\arzua\AppData\Local\Temp\ipykernel_7012\568598159.py:13: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  datos= datos.resample('H', closed='left', label='right').mean()


¿Hay valores vacíos en el dataset? True
¿Hay valores vacíos en el dataset después de completar? False


C:\Users\arzua\AppData\Local\Temp\ipykernel_7012\568598159.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  datos = datos.fillna(method='ffill').fillna(method='bfill')
C:\Users\arzua\AppData\Local\Temp\ipykernel_7012\568598159.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  datos = datos.fillna(method='ffill').fillna(method='bfill')


Tamaño del conjunto de entrenamiento: 18407
Tamaño del conjunto de validación: 3936
Tamaño del conjunto de prueba: 3936
Fecha fin entrenamiento: 2014-02-05 23:00:00
Fecha fin validación: 2014-07-19 23:00:00
                          Demand  Temperature  Holiday
Time                                                  
2014-02-05 19:00:00  6038.798486        29.60    False
2014-02-05 20:00:00  5700.070189        27.55    False
2014-02-05 21:00:00  5389.295345        25.55    False
2014-02-05 22:00:00  5191.951051        23.65    False
2014-02-05 23:00:00  4671.543949        22.95    False
                          Demand  Temperature  Holiday
Time                                                  
2014-07-19 19:00:00  5647.680616        11.55    False
2014-07-19 20:00:00  5377.088537        11.15    False
2014-07-19 21:00:00  5135.699017        11.00    False
2014-07-19 22:00:00  4856.480564        10.90    False
2014-07-19 23:00:00  4660.067225        10.65    False
                       